In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import pickle
import datetime

In [2]:
## large dataset
# dataset = sparse.load_npz("data/simulation_for_ucp.npz")

# date_range = pd.date_range(start='2023-02-02', periods=dataset.shape[1], freq='D')

# dataset = pd.DataFrame(dataset.A, columns = date_range)
# dataset.head(5)

In [2]:
# small dataset
dataset = pd.read_pickle('data\simulations_for_ucp_procced_head_100.pickle')
dataset.head(5)

prediction_date,2023-02-02,2023-02-03,2023-02-04,2023-02-05,2023-02-06,2023-02-07,2023-02-08,2023-02-09,2023-02-10,2023-02-11,...,2023-07-22,2023-07-23,2023-07-24,2023-07-25,2023-07-26,2023-07-27,2023-07-28,2023-07-29,2023-07-30,2023-07-31
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
00007e1b-ae0a-3989-b3f1-759732362837,0,1,1,1,1,0,1,0,1,1,...,1,1,1,1,0,1,1,0,1,1
0000ca52-4ef7-303a-a2ca-688a36e79620,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001f41c-cda8-4ea9-a349-65fd3441581e,1,0,1,1,0,1,1,0,1,1,...,1,1,1,1,1,1,1,0,1,1
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0


In [ ]:
# full dataset
provider_ids = pickle.load(open("./provider_ids.pickle", "rb"))
sparse_matrix = pickle.load(open("simulation_for_ucp.pickle", "rb")).A

pd.DataFrame(sparse_matrix, index = provider_ids.index)

In [3]:
#X = pickle.load(open("data\simulation_for_ucp_has_txn.pickle", "rb"))
#X = pd.read_pickle('data\simulation_for_ucp_has_txn.pickle')
provider_ids = pickle.load(open("data\provider_ids.pickle", "rb"))

In [4]:
# X = pd.read_pickle('data\simulation_for_ucp_has_txn.pickle')
# date_range = pd.date_range(start='2023-02-02', periods=dataset.shape[1], freq='D')

# X = pd.DataFrame(X.A, columns = date_range)
# X.head(5)

In [5]:
N_USERS = dataset.shape[0]
N_DAYS = dataset.shape[1]

In [6]:
def create_is_effective_crawl_df(data, crawl):
    is_effective_crawl = np.empty((N_USERS, N_DAYS))
    is_effective_crawl[:] = np.nan
    for i in range(N_USERS):
        one_slice_data = data.iloc[i]
        one_slice_crawl = crawl[i]
        buga = one_slice_data.cumsum()[one_slice_crawl]
        is_effective_crawl[i, one_slice_crawl] = np.diff(buga, prepend=0) >0
    
    return pd.DataFrame(is_effective_crawl, columns=data.columns)

In [7]:
def calc_days_with_updates(data, time_slot, exclusion_window=7):
    # data contain the relevant history data for certain date
    columns_in_slot = data.columns[-time_slot:- exclusion_window]
    return pd.DataFrame(data[columns_in_slot].sum(axis=1)/(time_slot-exclusion_window), columns=[f'n_updates_{time_slot}d'])

In [8]:
def calc_time_between_updates(data, time_slot, exclusion_window=7):
    # data contain the relevant history data for certain date
    columns_in_slot = data.columns[-time_slot:- exclusion_window]
    index_df = data[columns_in_slot].apply(lambda row: row[row == 1].index, axis=1)
    mean_list, max_list, min_list = [],[],[]
    for updates_dates in index_df:
        updates_dates_indexes = [data.columns.get_loc(date) for date in updates_dates]
        xdiff = [updates_dates_indexes[n]-updates_dates_indexes[n-1] for n in range(1,len(updates_dates_indexes))]        
        if xdiff == []:
            mean_list.append(0)
            max_list.append(0)
            min_list.append(0)
        else:
            mean_list.append(np.mean(xdiff))
            max_list.append(np.max(xdiff))
            min_list.append(np.min(xdiff))
            
    return pd.DataFrame({f'mean_time_{time_slot}d': mean_list, f'max_time_{time_slot}d': max_list, f'min_time_{time_slot}d': min_list}, index=data.index)
        

In [170]:
start_date_column = 120  # Adjust this to your desired starting column
# Get the column names (dates) from the dataset
date_columns = dataset.columns[start_date_column:]
# create fetures dataset
features_df = pd.melt(dataset[date_columns], ignore_index=False, value_name='has_txn')

time_slot = [30, 60, 90, 120]
for slot in time_slot:
    n_update_col = f'n_updates_{slot}d'
    mean_col = f'mean_time_{slot}d'
    max_col = f'max_time_{slot}d'
    min_col = f'min_time_{slot}d'
    cols_list = [n_update_col, mean_col, max_col, min_col]
    for col_name in cols_list:
        features_df[col_name] = 0
    for date in date_columns:
        end_index = dataset.columns.get_loc(date)
        start_index = end_index - start_date_column 
        columns_in_slot = dataset.columns[start_index:end_index]
        date_data = dataset[columns_in_slot]
        features_df.loc[features_df['prediction_date'] == date,n_update_col] = calc_days_with_updates(date_data, slot)
        features_df.loc[features_df['prediction_date'] == date, [mean_col, max_col, min_col]] = calc_time_between_updates(date_data, slot)
features_df

,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,n_updates_90d,mean_time_90d,max_time_90d,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d
credentialset_id,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.000000,8,1,0.253012,3.750000,12,1,0.221239,4.208333,14,1
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,1,0.826087,1.222222,2,1,0.792453,1.243902,3,1,0.819277,1.223881,3,1,0.805310,1.233333,3,1
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0,0.000000,0.000000,0,0,0.037736,6.000000,6,6,0.036145,10.000000,14,6,0.044248,16.500000,42,4
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,1,0.434783,2.333333,6,1,0.509434,2.000000,7,1,0.542169,1.840909,7,1,0.522124,1.931034,7,1
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,0,0.000000,0.000000,0,0,0.037736,12.000000,12,12,0.024096,12.000000,12,12,0.026549,37.500000,63,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0,0.043478,0.000000,0,0,0.075472,9.333333,18,3,0.060241,12.250000,21,3,0.070796,12.571429,26,1
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,0,0.478261,2.000000,4,1,0.509434,1.923077,6,1,0.542169,1.818182,6,1,0.548673,1.770492,6,1
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,0,0.391304,2.250000,5,1,0.396226,2.500000,6,1,0.421687,2.352941,8,1,0.380531,2.547619,8,1


# Crawling based features: 
- for the last 7 days
- target

In [10]:
np.random.seed(33)
crawl = np.random.binomial(1, 0.5, size=(N_USERS, N_DAYS)).astype(bool)
crawl_df = create_is_effective_crawl_df(dataset, crawl)
crawl_df = crawl_df.set_index(dataset.index)

In [11]:
crawl_df.head(5)

prediction_date,2023-02-02,2023-02-03,2023-02-04,2023-02-05,2023-02-06,2023-02-07,2023-02-08,2023-02-09,2023-02-10,2023-02-11,...,2023-07-22,2023-07-23,2023-07-24,2023-07-25,2023-07-26,2023-07-27,2023-07-28,2023-07-29,2023-07-30,2023-07-31
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0
00007e1b-ae0a-3989-b3f1-759732362837,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,NaN,1.0,1.0,0.0,NaN,NaN
0000ca52-4ef7-303a-a2ca-688a36e79620,NaN,NaN,NaN,NaN,1.0,NaN,1.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN
0001f41c-cda8-4ea9-a349-65fd3441581e,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,NaN,1.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,1.0,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [12]:
crawl_df.T.describe()

credentialset_id,0000349e-0a0f-3373-a4f1-05bce8b528b3,00007e1b-ae0a-3989-b3f1-759732362837,0000ca52-4ef7-303a-a2ca-688a36e79620,0001f41c-cda8-4ea9-a349-65fd3441581e,0001f720-64bc-361a-ba9f-5a21cbfbcbcb,00020524-e218-4e9a-adfb-b513bfced8a5,000257dd-99db-4996-a96e-43033d38a43b,00027fee-20d8-4880-8696-db72f085ac7e,0002c279-51eb-4cef-94c4-871d5c2a22ba,00034514-c589-4d8d-a4ed-f2563448030f,...,0017d192-1056-4135-ae6a-436af23199c6,0017d3ae-fd9b-3ea2-851c-47a8f4e10724,0018289a-fd6e-37dd-8501-9427a52f4d26,001838b6-2a54-4c08-a1c1-458fb20b5594,00185005-d0d3-4aff-95a5-e737e6974cb5,00187410-3f06-30db-a9e6-97c43ab763d0,00189d2c-d177-4ace-83bb-6e5807b73ef0,0018cd3e-e488-4548-8a95-f251b89a9b80,0018d6bb-5ee1-4c82-9f84-4975ab66d1e9,0018df90-340f-4926-ba6c-ff3b0d3fe325
count,87.000000,101.000000,98.000000,88.000000,98.000000,90.0,94.000000,79.000000,85.000000,95.000000,...,93.000000,89.000000,95.000000,91.000000,82.000000,89.000000,86.000000,88.000000,96.0,87.000000
mean,0.367816,0.851485,0.051020,0.715909,0.112245,0.0,0.117021,0.367089,0.023529,0.136842,...,0.150538,0.303371,0.621053,0.120879,0.792683,0.134831,0.779070,0.613636,0.0,0.597701
std,0.485006,0.357383,0.221171,0.453565,0.317290,0.0,0.323169,0.485091,0.152477,0.345504,...,0.359536,0.462319,0.487699,0.327793,0.407879,0.343479,0.417307,0.489706,0.0,0.493204
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000
50%,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,1.000000
75%,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


In [13]:
crawl_df.T.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 2023-02-02 to 2023-07-31
Data columns (total 100 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   0000349e-0a0f-3373-a4f1-05bce8b528b3  87 non-null     float64
 1   00007e1b-ae0a-3989-b3f1-759732362837  101 non-null    float64
 2   0000ca52-4ef7-303a-a2ca-688a36e79620  98 non-null     float64
 3   0001f41c-cda8-4ea9-a349-65fd3441581e  88 non-null     float64
 4   0001f720-64bc-361a-ba9f-5a21cbfbcbcb  98 non-null     float64
 5   00020524-e218-4e9a-adfb-b513bfced8a5  90 non-null     float64
 6   000257dd-99db-4996-a96e-43033d38a43b  94 non-null     float64
 7   00027fee-20d8-4880-8696-db72f085ac7e  79 non-null     float64
 8   0002c279-51eb-4cef-94c4-871d5c2a22ba  85 non-null     float64
 9   00034514-c589-4d8d-a4ed-f2563448030f  95 non-null     float64
 10  0003d36a-b0f8-4b1d-9b73-e22c80e3839b  87 non-null     float64
 11  0003f7c

In [14]:
def calc_days_since_last_crawl(row, current_date):
    if len(row[row>=0]) == 0:
        return np.nan
    date_str = row[row >= 0].index[-1]
    last_crawl_date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    current_date = datetime.datetime.strptime(current_date, '%Y-%m-%d')
    delta = current_date - last_crawl_date
    return delta.days

In [15]:
date_columns = dataset.columns[start_date_column:]
last_days_featues_df = pd.melt(dataset[date_columns], ignore_index=False, value_name='has_txn')
last_days_featues_df[['n_successful_crawls', 'n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']] = 0
for date in date_columns:
    end_index = dataset.columns.get_loc(date)
    start_index = end_index - 7 
    columns_in_slot = dataset.columns[start_index:end_index]
    last_days_df = crawl_df[columns_in_slot]
    #last_days_featues_df[['n_successful_crawls', 'n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']] = 0
    last_days_featues_df.loc[last_days_featues_df['prediction_date'] == date, 
                             'n_successful_crawls'] = pd.DataFrame(last_days_df[last_days_df == 1].count(axis=1),
                                                                   columns=['n_successful_crawls'])
    last_days_featues_df.loc[last_days_featues_df['prediction_date'] == date, 
                             'n_unsuccessful_crawls'] = pd.DataFrame(last_days_df[last_days_df == 0].count(axis=1),
                                                                     columns=['n_unsuccessful_crawls'])
    last_days_featues_df.loc[last_days_featues_df['prediction_date'] == date, 
                             'is_last_crawl_succed'] = pd.DataFrame(last_days_df.apply(
        lambda row: row[row>=0][-1] if len(row[row>=0])>0 else np.nan , axis=1), columns=['is_last_crawl_succed'])
    last_days_featues_df.loc[last_days_featues_df['prediction_date'] == date, 
                             'n_days_since_last_crawl'] = pd.DataFrame(last_days_df.apply(calc_days_since_last_crawl, axis=1, current_date = date), 
                                                                        columns=['n_days_since_last_crawl'])
last_days_featues_df = last_days_featues_df.drop('has_txn', axis=1)
last_days_featues_df

,prediction_date,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl
credentialset_id,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,3,3,0.0,1.0
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,2,3,1.0,2.0
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0,5,0.0,1.0
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,2,0,1.0,2.0
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,1,2,0.0,1.0
...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0,5,0.0,1.0
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,3,0,1.0,1.0
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,3,3,0.0,1.0


In [164]:
date_columns = dataset.columns[start_date_column:]
# features_df = pd.melt(dataset[date_columns], ignore_index=False, value_name='has_txn')
# features_df[['n_successful_crawls', 'n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']] = 0
for date in date_columns:
    end_index = dataset.columns.get_loc(date)
    start_index = end_index - 7 
    columns_in_slot = dataset.columns[start_index:end_index]
    last_days_df = crawl_df[columns_in_slot]
    #features_df[['n_successful_crawls', 'n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']] = 0
    features_df.loc[features_df['prediction_date'] == date, 
                             'n_successful_crawls'] = pd.DataFrame(last_days_df[last_days_df == 1].count(axis=1),
                                                                   columns=['n_successful_crawls'])
    features_df.loc[features_df['prediction_date'] == date, 
                             'n_unsuccessful_crawls'] = pd.DataFrame(last_days_df[last_days_df == 0].count(axis=1),
                                                                     columns=['n_unsuccessful_crawls'])
    features_df.loc[features_df['prediction_date'] == date, 
                             'is_last_crawl_succed'] = pd.DataFrame(last_days_df.apply(
        lambda row: row[row>=0][-1] if len(row[row>=0])>0 else np.nan , axis=1), columns=['is_last_crawl_succed'])
    features_df.loc[features_df['prediction_date'] == date, 
                             'n_days_since_last_crawl'] = pd.DataFrame(last_days_df.apply(calc_days_since_last_crawl, axis=1, current_date = date), 
                                                                        columns=['n_days_since_last_crawl'])
features_df = features_df.drop('has_txn', axis=1)
features_df


,prediction_date,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,n_updates_90d,...,max_time_90d,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0.260870,4.200000,8,2,0.301887,3.000000,8,1,0.253012,...,12,1,0.221239,4.208333,14,1,3.0,3.0,0.0,1.0
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,0.826087,1.222222,2,1,0.792453,1.243902,3,1,0.819277,...,3,1,0.805310,1.233333,3,1,2.0,3.0,1.0,2.0
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0.000000,0.000000,0,0,0.037736,6.000000,6,6,0.036145,...,14,6,0.044248,16.500000,42,4,0.0,5.0,0.0,1.0
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,0.434783,2.333333,6,1,0.509434,2.000000,7,1,0.542169,...,7,1,0.522124,1.931034,7,1,2.0,0.0,1.0,2.0
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,0.000000,0.000000,0,0,0.037736,12.000000,12,12,0.024096,...,12,12,0.026549,37.500000,63,12,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0.043478,0.000000,0,0,0.075472,9.333333,18,3,0.060241,...,21,3,0.070796,12.571429,26,1,0.0,5.0,0.0,1.0
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,0.478261,2.000000,4,1,0.509434,1.923077,6,1,0.542169,...,6,1,0.548673,1.770492,6,1,3.0,0.0,1.0,1.0
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,0.391304,2.250000,5,1,0.396226,2.500000,6,1,0.421687,...,8,1,0.380531,2.547619,8,1,3.0,3.0,0.0,1.0


In [165]:
features_df[['n_successful_crawls','n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']].describe()

,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl
count,6000.000000,6000.000000,5949.000000,5949.000000
mean,1.278167,2.207667,0.363759,1.948899
std,1.313549,1.603320,0.481121,1.285718
min,0.000000,0.000000,0.000000,1.000000
25%,0.000000,1.000000,0.000000,1.000000
50%,1.000000,2.000000,0.000000,1.000000
75%,2.000000,3.000000,1.000000,2.000000
max,7.000000,7.000000,1.000000,7.000000


In [16]:
last_days_featues_df.describe()

,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl
count,6000.000000,6000.000000,5949.000000,5949.000000
mean,1.278167,2.207667,0.363759,1.948899
std,1.313549,1.603320,0.481121,1.285718
min,0.000000,0.000000,0.000000,1.000000
25%,0.000000,1.000000,0.000000,1.000000
50%,1.000000,2.000000,0.000000,1.000000
75%,2.000000,3.000000,1.000000,2.000000
max,7.000000,7.000000,1.000000,7.000000


In [139]:
pd.concat([features_df, last_days_featues_df], axis=1)

,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,...,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,prediction_date,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.000000,8,1,...,1,0.221239,4.208333,14,1,2023-06-02,3,3,0.0,1.0
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,1,0.826087,1.222222,2,1,0.792453,1.243902,3,1,...,1,0.805310,1.233333,3,1,2023-06-02,2,3,1.0,2.0
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0,0.000000,0.000000,0,0,0.037736,6.000000,6,6,...,6,0.044248,16.500000,42,4,2023-06-02,0,5,0.0,1.0
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,1,0.434783,2.333333,6,1,0.509434,2.000000,7,1,...,1,0.522124,1.931034,7,1,2023-06-02,2,0,1.0,2.0
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,0,0.000000,0.000000,0,0,0.037736,12.000000,12,12,...,12,0.026549,37.500000,63,12,2023-06-02,1,2,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0,0.043478,0.000000,0,0,0.075472,9.333333,18,3,...,3,0.070796,12.571429,26,1,2023-07-31,0,5,0.0,1.0
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,0,0.478261,2.000000,4,1,0.509434,1.923077,6,1,...,1,0.548673,1.770492,6,1,2023-07-31,3,0,1.0,1.0
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,0,0.391304,2.250000,5,1,0.396226,2.500000,6,1,...,1,0.380531,2.547619,8,1,2023-07-31,3,3,0.0,1.0


# Target - is a crawl 'today' will succed

In [18]:
# np.random.seed(33)
# crawl = np.random.binomial(1, 0.5, size=(N_USERS, N_DAYS)).astype(bool)

In [19]:
date_columns = dataset.columns[start_date_column:]
target_df = pd.melt(dataset[date_columns], ignore_index=False, value_name='has_txn')
target_df['target'] = 0
for date in date_columns:
    date_index = dataset.columns.get_loc(date)
    fixed_crawl = crawl.copy()
    for i in range(len(fixed_crawl)):
        fixed_crawl[i][date_index] = True 
    crawl_df = create_is_effective_crawl_df(dataset, fixed_crawl)
    crawl_df = crawl_df.set_index(dataset.index)
    target_df.loc[target_df['prediction_date'] == date, 'target'] = crawl_df[date]
target_df

,prediction_date,has_txn,target
credentialset_id,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,1,1
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0,0
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,1,1
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,0,0
...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0,0
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,0,0
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,0,0


# Provider based features

In [20]:
provider_ids

credentialset_id
0000349e-0a0f-3373-a4f1-05bce8b528b3    867c4a38-0d75-46ff-a4c2-f44cb63be2c9
00007e1b-ae0a-3989-b3f1-759732362837    280a19b6-a767-4e7f-b6b4-e422d6876897
0001f41c-cda8-4ea9-a349-65fd3441581e    89e6e230-1333-448b-9800-dc2d09a85e99
00020524-e218-4e9a-adfb-b513bfced8a5    63260486-7618-4660-9445-7e4885e99042
000257dd-99db-4996-a96e-43033d38a43b    50761787-31da-4b17-8048-c6063b614b80
                                                        ...                 
fffdda46-1b19-352a-874e-81dc4ac19f05    862ccdd4-5e8e-4238-be44-f1e84e12b017
fffe4c1c-ba1b-4ad2-8a21-e248a141b47d    75f8d952-be81-4e22-951e-41b291d211f2
fffe75c9-bbd1-41c8-91fb-65e35bb4c52c    99382634-1f3d-45b5-9841-109cb0b1a4fc
fffebfdb-c0ff-466e-9907-bf7a9785bbd1    9638be39-8160-4fa6-8093-1434234885cb
ffff47bf-d149-4f6e-afbb-7db22fe81cd0    144e4d72-561b-41e0-b18b-4206de2e1c36
Name: provider_id, Length: 240297, dtype: object

In [21]:
dataset_with_provider = dataset.join(provider_ids)

In [77]:
def calc_n_updates_days(data):
    data = data.to_numpy()
    n_updates = data.sum()
    n_days = data.shape[0]*data.shape[1]
    return n_updates/n_days
    

In [82]:
weekday0 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 0]
weekday1 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 1]
weekday2 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 2]
weekday3 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 3]
weekday4 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 4]
weekday5 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 5]
weekday6 = [date for date in dataset.columns if datetime.datetime.strptime(date, '%Y-%m-%d').weekday() == 6]
weekday_list = [weekday0, weekday1, weekday2, weekday3, weekday4, weekday5, weekday6]

In [50]:
provider_feature_df = pd.DataFrame(index=dataset_with_provider['provider_id'].unique())
provider_feature_df

""
867c4a38-0d75-46ff-a4c2-f44cb63be2c9
280a19b6-a767-4e7f-b6b4-e422d6876897
NaN
89e6e230-1333-448b-9800-dc2d09a85e99
63260486-7618-4660-9445-7e4885e99042
50761787-31da-4b17-8048-c6063b614b80
50b5a2d8-06d1-4d8d-b8b3-2b945e7c35d6
a12f6956-9aca-4550-8caa-2e2f9532674c
9b9f5ceb-dea2-49db-8476-4530c1b0c92d
22f309a8-1746-49a7-bf6c-678729834573


In [194]:
features_df_with_provider = features_df.join(provider_ids)
features_df_with_provider.head()

,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,...,mean_time_90d,max_time_90d,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,provider_n_updates,n_updates_weekday,provider_id
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.0,8,1,...,3.75,12,1,0.221239,4.208333,14,1,0.349653,NaN,867c4a38-0d75-46ff-a4c2-f44cb63be2c9
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-03,0,0.217391,4.750000,8,3,0.301887,3.0,8,1,...,3.75,12,1,0.221239,4.208333,14,1,0.349653,NaN,867c4a38-0d75-46ff-a4c2-f44cb63be2c9
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-04,0,0.217391,4.750000,8,3,0.301887,3.0,8,1,...,3.75,12,1,0.221239,4.208333,14,1,0.349653,NaN,867c4a38-0d75-46ff-a4c2-f44cb63be2c9
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-05,0,0.173913,5.333333,8,3,0.301887,3.0,8,1,...,3.75,12,1,0.221239,4.208333,14,1,0.349653,NaN,867c4a38-0d75-46ff-a4c2-f44cb63be2c9
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-06,0,0.173913,5.333333,8,3,0.301887,3.0,8,1,...,3.75,12,1,0.221239,4.208333,14,1,0.349653,NaN,867c4a38-0d75-46ff-a4c2-f44cb63be2c9


In [132]:
weekday_dict = {}
for i, weekday in enumerate(weekday_list):
    weekday_data = dataset[weekday]
    weekday_dict[i] = calc_n_updates_days(weekday_data)
weekday_dict

{0: 0.2592307692307692,
 1: 0.3468,
 2: 0.3132,
 3: 0.3180769230769231,
 4: 0.3515384615384615,
 5: 0.10346153846153847,
 6: 0.17153846153846153}

In [155]:
mask = dataset_with_provider['provider_id'] == '867c4a38-0d75-46ff-a4c2-f44cb63be2c9'


In [172]:
provider_feature_df = pd.DataFrame(index=dataset_with_provider['provider_id'].unique())
#provider_feature_df[['provider_n_updates', 'provider_n_updates_day0', 'provider_n_updates_day1', 'provider_n_updates_day2', 
#                     'provider_n_updates_day3', 'provider_n_updates_day4', 'provider_n_updates_day5', 'provider_n_updates_day6']] = 0
for provider in dataset_with_provider['provider_id'].unique():
    mask_provider = dataset_with_provider['provider_id'] == provider
    provider_data = dataset_with_provider[mask_provider].drop('provider_id', axis=1)
    n_update_provider = calc_n_updates_days(provider_data)
    provider_feature_df.loc[provider, 'provider_n_updates'] = n_update_provider
    features_df_with_provider.loc[mask_provider, 'provider_n_updates'] = n_update_provider
    features_df_with_provider.loc[mask_provider, 'provider_n_updates'] = n_update_provider
    for i, weekday in enumerate(weekday_list):
        weekday_data = provider_data[weekday]
        mask_weekday = features_df_with_provider['prediction_date'].isin(weekday)
        n_update_weekday = calc_n_updates_days(weekday_data)
        provider_feature_df.loc[provider, f'provider_n_updates_weekday{i}'] = n_update_weekday
        features_df_with_provider.loc[mask_weekday & mask_provider, 'provider_n_updates_weekday'] = n_update_weekday
        features_df1.loc[mask_weekday & mask_provider, 'provider_n_updates_weekday'] = n_update_weekday
        features_df_with_provider.loc[mask_weekday, 'n_updates_weekday'] = weekday_dict[i]
features_df_with_provider

IndexError: boolean index did not match indexed array along dimension 0; dimension is 6000 but corresponding boolean dimension is 360000

In [195]:
provider_feature_df = pd.DataFrame(index=dataset_with_provider['provider_id'].unique())
#provider_feature_df[['provider_n_updates', 'provider_n_updates_day0', 'provider_n_updates_day1', 'provider_n_updates_day2', 
#                     'provider_n_updates_day3', 'provider_n_updates_day4', 'provider_n_updates_day5', 'provider_n_updates_day6']] = 0
for provider in dataset_with_provider['provider_id'].unique():
    mask_provider = dataset_with_provider['provider_id'] == provider
    provider_data = dataset_with_provider[mask_provider].drop('provider_id', axis=1)
    n_update_provider = calc_n_updates_days(provider_data)
    provider_feature_df.loc[provider, 'provider_n_updates'] = n_update_provider
    features_df_with_provider.loc[mask_provider, 'provider_n_updates'] = n_update_provider
    features_df.loc[mask_provider, 'provider_n_updates'] = n_update_provider
    for i, weekday in enumerate(weekday_list):
        weekday_data = provider_data[weekday]
        mask_weekday = features_df_with_provider['prediction_date'].isin(weekday)
        n_update_weekday = calc_n_updates_days(weekday_data)
        provider_feature_df.loc[provider, f'provider_n_updates_day{i}'] = n_update_weekday
        features_df_with_provider.loc[mask_weekday & mask_provider, 'provider_n_updates_weekday'] = n_update_weekday
        #features_df_with_provider.loc[mask_weekday & mask_provider, 'provider_n_updates_weekday'] = n_update_weekday
        features_df_with_provider.loc[mask_weekday, 'n_updates_weekday'] = n_update_weekday
        #features_df.loc[mask_weekday, 'n_updates_weekday'] = n_update_weekday
features_df_with_provider

<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days
<ipython-input-77-09aa11728a2d>:5: RuntimeWarning: invalid value encountered in scalar divide
  return n_updates/n_days


,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,...,max_time_90d,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,provider_n_updates,n_updates_weekday,provider_id,provider_n_updates_weekday
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.730769,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.459135
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-03,0,0.217391,4.750000,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.000000,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.000000
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-04,0,0.217391,4.750000,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.115385,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.314904
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-05,0,0.173913,5.333333,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.576923,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.375000
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-06,0,0.173913,5.333333,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.560000,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.472500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-27,0,0.304348,2.166667,5,1,0.396226,2.400000,9,1,...,9,1,0.460177,2.176471,9,1,0.444444,0.576923,88355d10-e310-465b-a173-953dacf0b96d,0.576923
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-28,0,0.304348,2.166667,5,1,0.396226,2.400000,9,1,...,9,1,0.451327,2.200000,9,1,0.444444,0.730769,88355d10-e310-465b-a173-953dacf0b96d,0.730769
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-29,0,0.347826,2.285714,5,1,0.415094,2.428571,9,1,...,9,1,0.451327,2.240000,9,1,0.444444,0.000000,88355d10-e310-465b-a173-953dacf0b96d,0.000000


In [184]:
features_df

,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,n_updates_90d,mean_time_90d,max_time_90d,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,provider_n_updates,n_updates_weekday
credentialset_id,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.000000,8,1,0.253012,3.750000,12,1,0.221239,4.208333,14,1,0.349653,NaN
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,1,0.826087,1.222222,2,1,0.792453,1.243902,3,1,0.819277,1.223881,3,1,0.805310,1.233333,3,1,0.800000,NaN
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0,0.000000,0.000000,0,0,0.037736,6.000000,6,6,0.036145,10.000000,14,6,0.044248,16.500000,42,4,NaN,NaN
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,1,0.434783,2.333333,6,1,0.509434,2.000000,7,1,0.542169,1.840909,7,1,0.522124,1.931034,7,1,0.555556,NaN
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,0,0.000000,0.000000,0,0,0.037736,12.000000,12,12,0.024096,12.000000,12,12,0.026549,37.500000,63,12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0,0.043478,0.000000,0,0,0.075472,9.333333,18,3,0.060241,12.250000,21,3,0.070796,12.571429,26,1,0.066667,NaN
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,0,0.478261,2.000000,4,1,0.509434,1.923077,6,1,0.542169,1.818182,6,1,0.548673,1.770492,6,1,0.427778,NaN
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,0,0.391304,2.250000,5,1,0.396226,2.500000,6,1,0.421687,2.352941,8,1,0.380531,2.547619,8,1,0.400000,NaN


In [196]:
features_df_with_provider#[mask_provider & mask_weekday]

,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,...,max_time_90d,min_time_90d,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,provider_n_updates,n_updates_weekday,provider_id,provider_n_updates_weekday
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.730769,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.459135
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-03,0,0.217391,4.750000,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.000000,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.000000
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-04,0,0.217391,4.750000,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.115385,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.314904
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-05,0,0.173913,5.333333,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.576923,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.375000
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-06,0,0.173913,5.333333,8,3,0.301887,3.000000,8,1,...,12,1,0.221239,4.208333,14,1,0.349653,0.560000,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.472500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-27,0,0.304348,2.166667,5,1,0.396226,2.400000,9,1,...,9,1,0.460177,2.176471,9,1,0.444444,0.576923,88355d10-e310-465b-a173-953dacf0b96d,0.576923
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-28,0,0.304348,2.166667,5,1,0.396226,2.400000,9,1,...,9,1,0.451327,2.200000,9,1,0.444444,0.730769,88355d10-e310-465b-a173-953dacf0b96d,0.730769
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-29,0,0.347826,2.285714,5,1,0.415094,2.428571,9,1,...,9,1,0.451327,2.240000,9,1,0.444444,0.000000,88355d10-e310-465b-a173-953dacf0b96d,0.000000


In [197]:
date_columns = dataset.columns[start_date_column:]
# features_df_with_provider = pd.melt(dataset[date_columns], ignore_index=False, value_name='has_txn')
# features_df_with_provider[['n_successful_crawls', 'n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']] = 0
for date in date_columns:
    end_index = dataset.columns.get_loc(date)
    start_index = end_index - 7 
    columns_in_slot = dataset.columns[start_index:end_index]
    last_days_df = crawl_df[columns_in_slot]
    #features_df_with_provider[['n_successful_crawls', 'n_unsuccessful_crawls', 'is_last_crawl_succed', 'n_days_since_last_crawl']] = 0
    features_df_with_provider.loc[features_df_with_provider['prediction_date'] == date, 
                             'n_successful_crawls'] = pd.DataFrame(last_days_df[last_days_df == 1].count(axis=1),
                                                                   columns=['n_successful_crawls'])
    features_df_with_provider.loc[features_df_with_provider['prediction_date'] == date, 
                             'n_unsuccessful_crawls'] = pd.DataFrame(last_days_df[last_days_df == 0].count(axis=1),
                                                                     columns=['n_unsuccessful_crawls'])
    features_df_with_provider.loc[features_df_with_provider['prediction_date'] == date, 
                             'is_last_crawl_succed'] = pd.DataFrame(last_days_df.apply(
        lambda row: row[row>=0][-1] if len(row[row>=0])>0 else np.nan , axis=1), columns=['is_last_crawl_succed'])
    features_df_with_provider.loc[features_df_with_provider['prediction_date'] == date, 
                             'n_days_since_last_crawl'] = pd.DataFrame(last_days_df.apply(calc_days_since_last_crawl, axis=1, current_date = date), 
                                                                        columns=['n_days_since_last_crawl'])
features_df_with_provider = features_df_with_provider.drop('has_txn', axis=1)
features_df_with_provider


,prediction_date,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,n_updates_90d,...,max_time_120d,min_time_120d,provider_n_updates,n_updates_weekday,provider_id,provider_n_updates_weekday,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0.260870,4.200000,8,2,0.301887,3.000000,8,1,0.253012,...,14,1,0.349653,0.730769,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.459135,3.0,3.0,0.0,1.0
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-03,0.217391,4.750000,8,3,0.301887,3.000000,8,1,0.253012,...,14,1,0.349653,0.000000,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.000000,3.0,4.0,0.0,1.0
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-04,0.217391,4.750000,8,3,0.301887,3.000000,8,1,0.253012,...,14,1,0.349653,0.115385,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.314904,2.0,5.0,0.0,1.0
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-05,0.173913,5.333333,8,3,0.301887,3.000000,8,1,0.253012,...,14,1,0.349653,0.576923,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.375000,2.0,5.0,0.0,1.0
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-06,0.173913,5.333333,8,3,0.301887,3.000000,8,1,0.253012,...,14,1,0.349653,0.560000,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0.472500,2.0,5.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-27,0.304348,2.166667,5,1,0.396226,2.400000,9,1,0.397590,...,9,1,0.444444,0.576923,88355d10-e310-465b-a173-953dacf0b96d,0.576923,1.0,0.0,1.0,2.0
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-28,0.304348,2.166667,5,1,0.396226,2.400000,9,1,0.385542,...,9,1,0.444444,0.730769,88355d10-e310-465b-a173-953dacf0b96d,0.730769,2.0,0.0,1.0,1.0
0018df90-340f-4926-ba6c-ff3b0d3fe325,2023-07-29,0.347826,2.285714,5,1,0.415094,2.428571,9,1,0.397590,...,9,1,0.444444,0.000000,88355d10-e310-465b-a173-953dacf0b96d,0.000000,2.0,1.0,0.0,1.0


In [187]:
mask_provider = dataset_with_provider['provider_id'] == provider
mask_weekday = features_df['prediction_date'].isin(weekday)
features_df[mask_weekday mask_provider]

<ipython-input-187-8e5ebc99465d>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  features_df[mask_weekday & mask_provider]


IndexError: indices are out-of-bounds

In [160]:
features_df1

,prediction_date,has_txn,n_updates_30d,mean_time_30d,max_time_30d,min_time_30d,n_updates_60d,mean_time_60d,max_time_60d,min_time_60d,...,n_updates_120d,mean_time_120d,max_time_120d,min_time_120d,prediction_date,n_successful_crawls,n_unsuccessful_crawls,is_last_crawl_succed,n_days_since_last_crawl,provider_n_updates
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,2023-06-02,0,0.260870,4.200000,8,2,0.301887,3.000000,8,1,...,0.221239,4.208333,14,1,2023-06-02,3,3,0.0,1.0,0.349653
00007e1b-ae0a-3989-b3f1-759732362837,2023-06-02,1,0.826087,1.222222,2,1,0.792453,1.243902,3,1,...,0.805310,1.233333,3,1,2023-06-02,2,3,1.0,2.0,0.800000
0000ca52-4ef7-303a-a2ca-688a36e79620,2023-06-02,0,0.000000,0.000000,0,0,0.037736,6.000000,6,6,...,0.044248,16.500000,42,4,2023-06-02,0,5,0.0,1.0,NaN
0001f41c-cda8-4ea9-a349-65fd3441581e,2023-06-02,1,0.434783,2.333333,6,1,0.509434,2.000000,7,1,...,0.522124,1.931034,7,1,2023-06-02,2,0,1.0,2.0,0.555556
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,2023-06-02,0,0.000000,0.000000,0,0,0.037736,12.000000,12,12,...,0.026549,37.500000,63,12,2023-06-02,1,2,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,2023-07-31,0,0.043478,0.000000,0,0,0.075472,9.333333,18,3,...,0.070796,12.571429,26,1,2023-07-31,0,5,0.0,1.0,0.066667
00189d2c-d177-4ace-83bb-6e5807b73ef0,2023-07-31,0,0.478261,2.000000,4,1,0.509434,1.923077,6,1,...,0.548673,1.770492,6,1,2023-07-31,3,0,1.0,1.0,0.427778
0018cd3e-e488-4548-8a95-f251b89a9b80,2023-07-31,0,0.391304,2.250000,5,1,0.396226,2.500000,6,1,...,0.380531,2.547619,8,1,2023-07-31,3,3,0.0,1.0,0.400000


In [142]:
features_df1 = pd.concat([features_df, last_days_featues_df], axis=1)
#pd.concat([features_df_with_provider, last_days_featues_df], axis=1)

In [162]:
provider_ids['0000ca52-4ef7-303a-a2ca-688a36e79620']

KeyError: '0000ca52-4ef7-303a-a2ca-688a36e79620'

In [5]:
# Define the starting column index for dates
start_date_column = 120  # Adjust this to your desired starting column

# Define the time slots and the exclusion window
time_slots = [30, 60, 90, 120]
exclusion_window = 7
start_date_column = 120  # Adjust this to your desired starting column
# Get the column names (dates) from the dataset
date_columns = dataset.columns[start_date_column:]

# Create a list to store calculated features as dictionaries
new_features_data = {}
 
# Iterate through each date column
for date_column in date_columns:
    #new_row = {}  # Dictionary to store calculated features for the current row
    
    # Calculate features for each time slot
    for slot in time_slots:
        # Calculate the end index for the time slot
        end_index = dataset.columns.get_loc(date_column)

        # Calculate the start index by subtracting the time slot duration and exclusion window
        start_index = end_index - slot 

        # Select the columns within the time slot (excluding the last 7 days)
        columns_in_slot = dataset.columns[start_index:end_index- exclusion_window]

        # Count positive values for the current row within the time slot
        feature_name = f'n_changes_{slot}d_{date_column}'
        new_features_data[feature_name] = dataset[columns_in_slot].sum(axis=1)

    # Add the calculated features for the current row to the list
    #new_features_data.append(new_row)

# Create a new DataFrame from the list of dictionaries
new_features_df = pd.DataFrame(new_features_data)

# Save the new features DataFrame to a CSV file
#new_features_df


In [252]:
import pandas as pd

# Create or load your original dataset
data = {
    'User_ID': ['User1', 'User2', 'User3'],
    '2023-09-01': [10, 20, 30],
    '2023-09-02': [15, 25, 35],
    '2023-09-03': [20, 30, 40]
}

original_df = pd.DataFrame(data)

# Melt the original DataFrame
melted_df = pd.melt(original_df, id_vars=['User_ID'], var_name='Date', value_name='Value')

# Calculate new columns based on the original dataset and add them to the melted dataset
melted_df['Double_Value'] = melted_df['Value'] * 2

# Pivot the melted DataFrame back to the desired format (user IDs as rows and dates as columns)
final_df = melted_df.pivot(index='User_ID', columns='Date', values=['Value', 'Double_Value'])

# Reset the column index to flatten the resulting DataFrame
final_df.reset_index(inplace=True)

# Display the final DataFrame
final_df


User_ID      Value                       Double_Value             \
Date         2023-09-01 2023-09-02 2023-09-03   2023-09-01 2023-09-02   
0      User1         10         15         20           20         30   
1      User2         20         25         30           40         50   
2      User3         30         35         40           60         70   

                 
Date 2023-09-03  
0            40  
1            60  
2            80

In [6]:
new_features_df

,n_changes_30d_2023-06-02,n_changes_60d_2023-06-02,n_changes_90d_2023-06-02,n_changes_120d_2023-06-02,n_changes_30d_2023-06-03,n_changes_60d_2023-06-03,n_changes_90d_2023-06-03,n_changes_120d_2023-06-03,n_changes_30d_2023-06-04,n_changes_60d_2023-06-04,...,n_changes_90d_2023-07-29,n_changes_120d_2023-07-29,n_changes_30d_2023-07-30,n_changes_60d_2023-07-30,n_changes_90d_2023-07-30,n_changes_120d_2023-07-30,n_changes_30d_2023-07-31,n_changes_60d_2023-07-31,n_changes_90d_2023-07-31,n_changes_120d_2023-07-31
credentialset_id,,,,,,,,,,,,,,,,,,,,,
0000349e-0a0f-3373-a4f1-05bce8b528b3,6,16,21,25,5,16,21,25,5,16,...,19,26,5,9,18,26,4,8,17,26
00007e1b-ae0a-3989-b3f1-759732362837,19,42,68,91,19,43,68,92,18,42,...,66,89,18,44,67,89,18,44,67,90
0000ca52-4ef7-303a-a2ca-688a36e79620,0,2,3,5,0,2,3,5,0,1,...,0,2,0,0,0,2,0,0,0,2
0001f41c-cda8-4ea9-a349-65fd3441581e,10,27,45,59,11,27,46,59,10,26,...,44,60,13,29,44,61,13,29,44,62
0001f720-64bc-361a-ba9f-5a21cbfbcbcb,0,2,2,3,1,3,3,4,1,3,...,5,7,4,4,5,7,4,4,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00187410-3f06-30db-a9e6-97c43ab763d0,1,4,5,8,1,4,5,8,1,4,...,5,8,1,4,5,8,1,4,5,8
00189d2c-d177-4ace-83bb-6e5807b73ef0,13,31,46,69,13,32,47,69,12,31,...,45,63,12,28,45,62,11,27,45,62
0018cd3e-e488-4548-8a95-f251b89a9b80,11,20,35,46,10,20,35,45,9,20,...,35,44,10,22,35,43,9,21,35,43


In [7]:
# Define the starting column index for dates
start_date_column = 120  # Adjust this to your desired starting column

# Define the time slots and the exclusion window
time_slots = [30, 60, 90, 120]
exclusion_window = 7

# Get the column names (dates) from the dataset
date_columns = dataset.columns[start_date_column:]

# Create a list to store calculated features as dictionaries
new_features_data= {}
 
# Iterate through each date column
for date_column in date_columns:
    new_row = {}  # Dictionary to store calculated features for the current row
    #new_features_data[date] = date_column
    # Calculate features for each time slot
    for slot in time_slots:
        # Calculate the end index for the time slot
        end_index = dataset.columns.get_loc(date_column)

        # Calculate the start index by subtracting the time slot duration and exclusion window
        start_index = end_index - slot 

        # Select the columns within the time slot (excluding the last 7 days)
        columns_in_slot = dataset.columns[start_index:end_index- exclusion_window]

        # Count positive values for the current row within the time slot
        feature_name = f'n_updates_{slot}d'
        new_row[feature_name] = dataset[columns_in_slot].sum(axis=1)/slot

    # Add the calculated features for the current row to the list
    new_features_data[date_column]= new_row

# Create a new DataFrame from the list of dictionaries
#new_features_df = pd.DataFrame(new_features_data).T

# Save the new features DataFrame to a CSV file
#new_features_df


In [10]:
dataset.columns = pd.to_datetime(dataset.columns)

# Calculate the differences between updates for different time slots
time_slots = [30, 60, 90, 120]
time_slot_results = {}

for slot in time_slots:
    time_slot_results[slot] = []
    for index, row in dataset.iterrows():
        update_dates = row[row == 1].index
        if len(update_dates) > 1:
            days_between_updates = [(update_dates[i + 1] - update_dates[i]).days for i in range(len(update_dates) - 1)]
            days_within_time_slot = [days for days in days_between_updates if days <= slot]
            time_slot_results[slot].extend(days_within_time_slot)

# Print the results
for slot, days_within_time_slot in time_slot_results.items():
    average_days = sum(days_within_time_slot) / len(days_within_time_slot) if len(days_within_time_slot) > 0 else 0
    print(f"Average days between updates within {slot} days: {average_days:.2f}")


Average days between updates within 30 days: 2.70
Average days between updates within 60 days: 2.89
Average days between updates within 90 days: 2.97
Average days between updates within 120 days: 2.97
